In [1]:
import os
import json
import subprocess
import sys
from pathlib import Path
from typing import List
import yaml
import glob
import shutil

from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from IPython.display import SVG, display

In [2]:
BASE_OUTPUT_DIR = Path("outputs")
VIDEO_DIR = BASE_OUTPUT_DIR / "videos"
AUDIO_DIR = BASE_OUTPUT_DIR / "audio"
FINAL_DIR = BASE_OUTPUT_DIR / "final"
TEMP_DIR = BASE_OUTPUT_DIR / "temp"

for folder in [BASE_OUTPUT_DIR, VIDEO_DIR, AUDIO_DIR, FINAL_DIR, TEMP_DIR]:
    folder.mkdir(parents=True, exist_ok=True)

# Adjust this path if your Manim outputs are located elsewhere.
MANIM_MEDIA_DIR = Path("media") / "videos" / "temp_manim_script" / "720p30"


In [14]:
# Load API keys
file_path = "../keys.json"
with open(file_path, "r", encoding="utf-8") as file:
    key_data = json.load(file)

os.environ["OPENAI_API_KEY"] = key_data["openai_api_key"]

# Set up the LLM
model_name = "gpt-4o-mini"
llm = ChatOpenAI(temperature=0, model_name=model_name)

In [4]:
def generate_yaml_outline(topic_text: str) -> List[str]:
    """
    Ask the LLM to create an outline for a Manim animation based on a topic,
    and output it as a YAML list. Each item represents a self-contained animatable step.
    
    The prompt ensures:
      - The first item is a title slide explicitly labeled as "Title Slide".
      - Each step is focused, independent, and suitable for 2D Manim animations.
    """
    prompt = f"""
Create a structured YAML list for a 2D Manim animation on the topic: **"{topic_text}"**.

- The **first item** must be a **title slide** labeled as: **"Title Slide: {topic_text}"**.
- Each following item should describe **one self-contained animation step**.
- Steps must be **clear, independent**, and **suitable for 2D animations only** (no 3D elements).
- Avoid vague descriptions and ensure each step **can be animated on its own**.
- **Important:** Ensure that any colons (:) in the YAML output are properly escaped or enclosed in quotes to avoid parsing errors.

Return only a **valid YAML list** without extra formatting or explanations.
"""
    response = llm.invoke(prompt)
    outline_yaml = response.content.strip()
    
    # Remove YAML code fences if present.
    if outline_yaml.startswith("```yaml"):
        outline_yaml = outline_yaml.split("```yaml", 1)[1]
    if outline_yaml.endswith("```"):
        outline_yaml = outline_yaml.rsplit("```", 1)[0]
    
    try:
        outline = yaml.safe_load(outline_yaml)
        if not isinstance(outline, list):
            raise ValueError("Outline is not a list")
    except Exception as e:
        raise ValueError(f"Failed to parse YAML outline: {e}")
    
    # Save the outline to a YAML file for interpretation.
    with open("animation_outline.yaml", "w", encoding="utf-8") as f:
        yaml.dump(outline, f)
    
    print("✅ YAML Outline saved to animation_outline.yaml")
    return outline


In [5]:
##############################################
# 2. Generate Manim Code for a Single Module  #
##############################################

def generate_manim_code_for_step(step_description: str) -> str:
    """
    Generate a self-contained Manim Python script for a single step.
    The output is raw Python code (no markdown fences or extra commentary).
    """
    prompt = f"""
You are an expert in Manim. Given the step description below,
produce a valid, self-contained, and runnable Manim Python script that implements this step.
Provide only the Python code (do not wrap your answer in markdown code fences or include extra commentary).

Step Description:
{step_description}

Your response should contain only the executable Python code.
"""
    response = llm.invoke(prompt)
    code = response.content.strip()
    if code.startswith("```python"):
        code = code.split("```python", 1)[1]
    if code.endswith("```"):
        code = code.rsplit("```", 1)[0]
    return code.strip()

In [6]:

##############################################
# 3. Iterative Debugging for a Manim Script    #
##############################################

def iterative_debug(code: str, full_error: str, max_attempts: int = 3) -> str:
    """
    Iteratively ask the LLM for a corrected version of the code based on the full error traceback.
    """
    current_code = code
    for attempt in range(1, max_attempts + 1):
        print(f"Full error traceback:\n{full_error}\n")
        prompt = f"""
Aim:
Fix the Manim code so that it runs correctly and generates the intended animation.

Current Code:
{current_code}

Full Error Traceback:
{full_error}
        
Please provide a corrected version of the above code that fixes the error.
Output only the raw Python code (no markdown or additional commentary).
"""
        response = llm.invoke(prompt, max_tokens=1500)
        corrected_code = response.content.strip()
        if corrected_code.startswith("```python"):
            corrected_code = corrected_code.split("```python", 1)[1]
        if corrected_code.endswith("```"):
            corrected_code = corrected_code.rsplit("```", 1)[0]
        current_code = corrected_code.strip()
        print(f"\nAttempt {attempt}: Updated Code\n{'-'*40}\n{current_code}\n{'-'*40}\n")
    return current_code


In [7]:

##############################################
# 4. Run a Manim Script                        #
##############################################

def run_manim_script(manim_code: str, output_filename: str) -> Path:
    """
    Write the Manim code to a temporary file, run it via subprocess,
    and then copy the generated video from the default Manim output directory
    to our designated VIDEO_DIR.
    
    Returns the new path to the copied video file.
    """
    temp_script_path = TEMP_DIR / "temp_manim_script.py"
    with open(temp_script_path, "w", encoding="utf-8") as f:
        f.write(manim_code)
    
    cmd = [
        sys.executable,
        "-m", "manim",
        str(temp_script_path),
        "-qm",
        "-o", output_filename
    ]
    try:
        subprocess.run(cmd, check=True)
        print(f"✅ Manim animation generated with base name: {output_filename}")
    except subprocess.CalledProcessError as e:
        raise Exception(f"Manim failed: {e}")
    
    # Construct expected path using glob search.
    pattern = str(MANIM_MEDIA_DIR / f"*{output_filename}")
    matching_files = glob.glob(pattern)
    if not matching_files:
        raise FileNotFoundError(f"Could not find generated video matching {pattern}")
    
    generated_video = Path(matching_files[0])
    dest_video = VIDEO_DIR / generated_video.name
    shutil.copy(generated_video, dest_video)
    print(f"✅ Copied video to: {dest_video}")
    return dest_video

In [8]:
##############################################
# 5. Generate Dialogue for a Step             #
##############################################

def generate_dialogue_for_step(step_description: str) -> str:
    """
    Generate a short narration line (1-2 sentences) for the step.
    Do not include any timestamps; provide only the voiceover text.
    """
    prompt = f"""
You are an expert in educational narration. Given the following step description,
provide a short narration line (1-2 sentences) for the step.
Do not include any timestamps or extraneous labels; just provide the voiceover text.

Step Description:
{step_description}
"""
    response = llm.invoke(prompt)
    dialogue_line = response.content.strip()
    return dialogue_line


In [9]:
##############################################
# 6. Text-to-Speech Conversion (TTS)           #
##############################################

from gtts import gTTS

def text_to_speech(text: str, output_audio: Path):
    """
    Convert the provided text to speech using gTTS and save as an MP3 file.
    """
    tts = gTTS(text=text, lang="en")
    tts.save(str(output_audio))
    print(f"Generated TTS audio saved to: {output_audio}")

In [10]:
import ffmpeg
import cv2
import numpy as np
from pathlib import Path

def get_duration(filepath: Path) -> float:
    """
    Returns the duration of a media file in seconds using ffmpeg.probe.
    """
    probe = ffmpeg.probe(str(filepath))
    return float(probe["format"]["duration"])

def is_last_frame_black(video_path: Path, threshold: float = 10) -> bool:
    """
    Checks if the last frame of the video is completely black.
    Uses OpenCV to grab the last frame and computes its average brightness.
    If the average brightness is below the threshold, we consider it black.
    """
    cap = cv2.VideoCapture(str(video_path))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames == 0:
        cap.release()
        return False
    # Move to the last frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, total_frames - 1)
    ret, frame = cap.read()
    cap.release()
    if not ret or frame is None:
        return False
    # Compute average brightness (in grayscale, brightness near 0 is black)
    avg_brightness = np.mean(frame)
    return avg_brightness < threshold

def combine_audio_and_video(video_path: Path, audio_path: Path, final_output_filename: str) -> Path:
    """
    Combines video and audio so that:
      - If the video is longer than the audio, the audio is padded with silence.
      - If the audio is longer than the video, the video is extended by freezing its last frame.
    
    Additionally, if the video’s last frame is completely black, then when the audio is longer,
    we trim (or extend) the video to exactly match the audio duration (thus “expanding” the final black frame).
    
    This function uses ffmpeg-python so no raw command line calls are made.
    """
    final_output = Path("outputs/final") / final_output_filename
    
    # Determine durations.
    video_duration = get_duration(video_path)
    audio_duration = get_duration(audio_path)
    
    # Check if the last frame is completely black.
    last_frame_black = is_last_frame_black(video_path)
    
    # Create input streams.
    video_in = ffmpeg.input(str(video_path))
    audio_in = ffmpeg.input(str(audio_path))
    
    # Choose filters based on durations and the content of the last frame.
    if audio_duration > video_duration:
        # Audio is longer than video → extend video.
        pad_duration = audio_duration - video_duration
        # Use tpad to extend the video by cloning the last frame.
        video_filtered = video_in.video.filter('tpad', stop_mode='clone', stop_duration=pad_duration)
        # No filter needed on audio.
        audio_filtered = audio_in.audio
    elif video_duration > audio_duration:
        if last_frame_black:
            # Video is longer than audio but the last frame is black.
            # In this case, trim the video to the audio duration (i.e. “expand” the final black frame
            # over the entire audio duration) rather than padding the audio with silence.
            video_filtered = video_in.video.filter('trim', duration=audio_duration).filter('setpts', 'PTS-STARTPTS')
            audio_filtered = audio_in.audio
        else:
            # Video is longer than audio and the last frame is not black,
            # so pad the audio with silence (apad) to match the video length.
            pad_duration = video_duration - audio_duration
            video_filtered = video_in.video
            audio_filtered = audio_in.audio.filter('apad', pad_dur=pad_duration)
    else:
        # Durations are equal.
        video_filtered = video_in.video
        audio_filtered = audio_in.audio

    # Build the output; re-encode video with libx264 and audio with AAC.
    out = ffmpeg.output(video_filtered, audio_filtered, str(final_output),
                        vcodec='libx264', acodec='aac', strict='experimental')
    
    # Run the process quietly.
    ffmpeg.run(out, quiet=True)
    
    return final_output


In [11]:
##############################################
# 8. Concatenate Multiple Video Clips         #
##############################################

def concatenate_videos(video_files: List[str], final_output: str):
    """
    Concatenate multiple video files into one final video using ffmpeg.
    """
    list_filename = "videos_to_concat.txt"
    with open(list_filename, "w", encoding="utf-8") as f:
        for vf in video_files:
            f.write(f"file '{os.path.abspath(vf)}'\n")
    cmd = [
        "ffmpeg", "-y",
        "-f", "concat",
        "-safe", "0",
        "-i", list_filename,
        "-c", "copy",
        final_output
    ]
    subprocess.run(cmd, check=True)
    print(f"✅ Final video created: {final_output}")


In [12]:
##############################################
# 9. Full Animation Pipeline (Step-by-Step)     #
##############################################

def generate_animation_pipeline(topic_text: str):
    """
    Complete pipeline:
      1. Generate a YAML outline for the topic.
      2. For each step in the outline:
         a. Generate Manim code and run it to produce a video clip.
         b. Generate a narration line for the step.
         c. Convert the narration to speech.
         d. Combine the video and the audio.
      3. Concatenate all the step videos into one final video.
    
    If any step fails, it logs a warning and skips that segment.
    """
    outline = generate_yaml_outline(topic_text)
    step_video_files = []
    
    for idx, step in enumerate(outline, start=1):
        print(f"\n========== Processing Step {idx}: {step} ==========\n")
        try:
            # Define filenames.
            step_video_filename = f"step_{idx}.mp4"
            step_audio_filename = f"step_{idx}.mp3"
            step_final_filename = f"step_{idx}_final.mp4"
            
            # Step 2a: Generate Manim code and run it.
            manim_code = generate_manim_code_for_step(step)
            video_path = run_manim_script(manim_code, output_filename=step_video_filename)
            
            # Step 2b: Generate narration.
            dialogue_line = generate_dialogue_for_step(step)
            print(f"Generated dialogue: {dialogue_line}")
            
            # Step 2c: Convert dialogue to speech.
            audio_path = AUDIO_DIR / step_audio_filename
            text_to_speech(dialogue_line, output_audio=audio_path)
            
            # Step 2d: Combine video and audio.
            final_video_path = combine_audio_and_video(video_path, audio_path, final_output_filename=step_final_filename)
            step_video_files.append(str(final_video_path))
            
        except Exception as e:
            print(f"Warning: Step {idx} failed ({str(e)}). Skipping this segment.")
            continue

    if step_video_files:
        final_video = "final_animation_video.mp4"
        concatenate_videos(step_video_files, final_output=final_video)
        print("\n=== Animation Pipeline Complete! ===\n")
    else:
        print("No valid segments were created. Pipeline finished with no output.")

In [13]:

if __name__ == "__main__":
    # For example, let's animate the concept of Torque.
    topic = "Pythagorean Theorem Definition, Formula, and How It Works."
    generate_animation_pipeline(topic)

✅ YAML Outline saved to animation_outline.yaml

========== Processing Step 1: Title Slide: Pythagorean Theorem Definition, Formula, and How It Works. ==========



<frozen runpy>:128: RuntimeWarning: 'manim.__main__' found in sys.modules after import of package 'manim', but prior to execution of 'manim.__main__'; this may result in unpredictable behaviour


Manim Community v0.19.0



[03/20/25 11:44:15] INFO     Animation 0 : Partial movie file written in '/home/siva/llm_proj/EduVision/notebooks/media/videos/temp_manim_script/720p30/partial_movie_files/PythagoreanTheoremTitleSlide/2016333726_3704264090_223132457.mp4'                                      ]8;id=738430;file:///home/siva/anaconda3/envs/edu/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=477355;file:///home/siva/anaconda3/envs/edu/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\


[03/20/25 11:44:16] INFO     Animation 1 : Partial movie file written in '/home/siva/llm_proj/EduVision/notebooks/media/videos/temp_manim_script/720p30/partial_movie_files/PythagoreanTheoremTitleSlide/543634251_2766220452_2983986657.mp4'                                      ]8;id=948971;file:///home/siva/anaconda3/envs/edu/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=78714;file:///home/siva/anaconda3/envs/edu/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
[03/20/25 11:44:17] INFO     Animation 2 : Partial movie file written in '/home/siva/llm_proj/EduVision/notebooks/media/videos/temp_manim_script/720p30/partial_movie_files/PythagoreanTheoremTitleSlide/543634251_2408024828_3870190898.mp4'                                      ]8;id=196175;file:///home/siva/anaconda3/envs/edu/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=745289;file:///home/siva/anacon

...

<frozen runpy>:128: RuntimeWarning: 'manim.__main__' found in sys.modules after import of package 'manim', but prior to execution of 'manim.__main__'; this may result in unpredictable behaviour


Manim Community v0.19.0



[03/20/25 11:45:43] INFO     Animation 0 : Partial movie file written in '/home/siva/llm_proj/EduVision/notebooks/media/videos/temp_manim_script/720p30/partial_movie_files/PythagoreanTheoremSummary/2016333726_617846104_223132457.mp4'                                          ]8;id=775689;file:///home/siva/anaconda3/envs/edu/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=475640;file:///home/siva/anaconda3/envs/edu/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                    INFO     Animation 1 : Partial movie file written in '/home/siva/llm_proj/EduVision/notebooks/media/videos/temp_manim_script/720p30/partial_movie_files/PythagoreanTheoremSummary/543634251_2408024828_3779399106.mp4'                                         ]8;id=657558;file:///home/siva/anaconda3/envs/edu/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=549611;file:///home/siva/anaco

ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

✅ Final video created: final_animation_video.mp4

=== Animation Pipeline Complete! ===



In [15]:
# Example usage to visualize the outline
topic_text = "Pythagorean Theorem Definition, Formula, and How It Works."
outline = generate_yaml_outline(topic_text)

print("Generated Animation Outline:")
print("-" * 40)
for idx, step in enumerate(outline, start=1):
    print(f"Step {idx}: {step}")
print("-" * 40)

NotFoundError: Error code: 404 - {'error': {'message': 'The model `o3-mini` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [21]:
# To store final video clip filenames.
step_video_files = []

In [22]:
for idx, step in enumerate(outline, start=1):
    print(f"\n========== Processing Step {idx}: {step} ==========\n")
    
    # Define filenames.
    step_video_filename = f"step_{idx}.mp4"
    step_audio_filename = f"step_{idx}.mp3"
    step_final_filename = f"step_{idx}_final.mp4"
    
    # Step 2a: Generate Manim code and run it.
    manim_code = generate_manim_code_for_step(step)
    video_path = run_manim_script(manim_code, output_filename=step_video_filename)
    
    # Step 2b: Generate narration.
    dialogue_line = generate_dialogue_for_step(step)
    print(f"Generated dialogue: {dialogue_line}")
    
    # Step 2c: Convert dialogue to speech.
    audio_path = AUDIO_DIR / step_audio_filename
    text_to_speech(dialogue_line, output_audio=audio_path)
    
    # Step 2d: Combine video and audio.
    final_video_path = combine_audio_and_video(video_path, audio_path, final_output_filename=step_final_filename)
    step_video_files.append(str(final_video_path))
    break


========== Processing Step 1: {'Title Slide': 'Torque in Physics: definition, formula, and how lever arms work.'} ==========



<frozen runpy>:128: RuntimeWarning: 'manim.__main__' found in sys.modules after import of package 'manim', but prior to execution of 'manim.__main__'; this may result in unpredictable behaviour
Animation 0: Write(Text('Torque in Physics')):   0%|                                                                                                                                                                                          | 0/60 [00:00<?, ?it/s]

Manim Community v0.19.0



[03/18/25 23:29:41] INFO     Animation 0 : Partial movie file written in '/home/siva/llm_proj/EduVision/notebooks/media/videos/temp_manim_script/720p30/partial_movie_files/TitleSlide/2016333726_2427665977_223132457.mp4'                   ]8;id=124061;file:///home/siva/anaconda3/envs/edu/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=470075;file:///home/siva/anaconda3/envs/edu/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\


[03/18/25 23:29:42] INFO     Animation 1 : Partial movie file written in '/home/siva/llm_proj/EduVision/notebooks/media/videos/temp_manim_script/720p30/partial_movie_files/TitleSlide/543634251_3588616603_3711886643.mp4'                   ]8;id=418640;file:///home/siva/anaconda3/envs/edu/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=104916;file:///home/siva/anaconda3/envs/edu/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
[03/18/25 23:29:43] INFO     Animation 2 : Partial movie file written in '/home/siva/llm_proj/EduVision/notebooks/media/videos/temp_manim_script/720p30/partial_movie_files/TitleSlide/543634251_2408024828_678225912.mp4'                    ]8;id=798013;file:///home/siva/anaconda3/envs/edu/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=222315;file:///home/siva/anaconda3/envs/edu/lib/python3.13/site-packages/manim/scene/scene_file_writer.p